In [1]:
clean_up = True
%run StdPackages.ipynb
d['gams'] = os.path.join(d['CGE'],'gams')

# Production module

The production module includes nested production functions defined over time, sectors, and goods. Durables can be accumulated using investment goods that are precured with adjustment costs.

## Example A: S symmetric production sectors

In [2]:
name = 'A'

Consider the case with $S$ symmetric production sectors. Assume that all sectors produce a unique output using a CES-like production function. The following creates the relevant nesting structure from this notebook:

### A.1: Create nesting tree

With $S$ production sectors producing unique outputs we have $N=S$ goods. Assuming that all goods are demanded by each sector, we can define one nesting tree as follows:

In [3]:
N=3 # number of sectors and goods
sectors = ['s'+str(i) for i in range(1,N+1)]
goods = ['n'+str(i) for i in range(1,N+1)]
t = [] # The nesting tree consists of tuples with (sector, knot of nest, branch of nest).
for i in range(N):
    t += [(sectors[i],goods[i],good+'_input') for good in goods];
tree = NestingTree.tree('t1', tree = t)
A_tree = NestingTree.AggTree(name, trees = {tree.name: tree})
A_tree(namespace = {n+'_input':n for n in goods})

Note that the nesting tree automatically includes a number of relevant features:
* A namespace, e.g. with names for the mapping in the relevant tree. (```self.ns```)
* A database with symbols (e.g. mapping, set of goods, sectors). (```self.db```)
* A name/specification of function type to be applied for each subet of the tree (```self.trees['t1'].f```)

Furthermore, note that it is best-practice to use the postfix '_input' for the input types, to distinguish between a 'n1' that is the production and 'n1' that is the input.

### A.2: Initialize other symbols

* **Time**: Define a set $t$, as well as simple subsets (t0,tE,txE,tx0E).
* **Decision variables:** 
    * Define variable $qD$ as the level of demand and $pD$ the corresponding price (incl. taxes). Defined over inputs and intermediate goods.
    * Define variable $qS$ as the level of suplpy and $pS$ the corresponding price (excl. taxes). Defined over outputs.
    * Inclusive value ```qiv[t,s,n]``` used for scale-preserving functions. Auxiliary variable used to define the denominator for share functions.
* **Technical parameters:**
    * Every knot in the nesting tree is characterized by an elasticity (or smoothing parameter),
    * Every branch in th nesting tree is characterized by a share parameter.
    * Normalizing term used for scale-preserving functions (a value of 0 is neutral) (NB: Defined as parameter).

In [4]:
db = A_tree.db
db['t'] = pd.Index([1,2], name='t')
# Standard subsets:
db['t0'] = db.get('t')[0:1]
db['tE'] = db.get('t')[-1:]
db['txE'] = rc_pd(db.get('t'), c = ('not', db.get('tE')))
db['tx0'] = rc_pd(db.get('t'), c = ('not', db.get('t0')))
db['tx0E'] = rc_pd(db.get('t'), c = ('not', ('or', [db['tE'],db['t0']])))
# Variables:
db['qS'] = pd.Series(1, index = MergeDomains([db.get('t'),A_tree.get('output')],db), name = 'qS')
db['pS'] = pd.Series(1, index = MergeDomains([db.get('t'),A_tree.get('output')],db), name = 'pS')
db['qD'] = pd.Series(1/N, index = MergeDomains([db.get('t'),A_tree.get('int').union(A_tree.get('input'))],db), name = 'qD')
db['pD'] = pd.Series(1, index = db['qD'].index, name = 'pD')
# Technical parameters (stored as variables to be able to endogenize them at some point):
db['sigma'] = pd.Series(0.5, index = A_tree.get('knot'), name = 'sigma')
db['mu'] = pd.Series(1/N, index = A_tree.get('map'), name = 'mu')
db['qnorm'] = Database.gpy(pd.Series(0, index = MergeDomains([db.get('t'),A_tree.get('knot')],db), name = 'qnorm'), **{'type':'parameter'})

*Write to a gdx file:*

In [5]:
db.merge_internal()
db_str = os.path.join(d['work'],name)
db.database.export(db_str)

### A.3: Define settings and model

Set up model:
* Define groups: We start defining the model without price wedges and installation costs, and only solve the model in baseline mode (no calibration). In this case, the endogenous variables are: Prices on intermediate goods + outputs, and quantities for everything but the supply variables.
* Initialize settings: Initialize with groups.
* Define blocks: Use nesting trees to write the relevant blocks of code.
* Write text
* Set up GmsModel and run.

*Groups:*

In [6]:
endo = GmsPy.Group(f"G_{name}_endo", v = [('pD', db['int_A']),
                                          ('pS', db['output_A']),
                                          ('qD', db['int_A']),
                                          ('qD', db['input_A'])])
exo  = GmsPy.Group(f"G_{name}_exo", v = [('mu', db['map_A']),
                                         ('sigma', db['knot_A']),
                                         ('pD', db['input_A']),
                                         ('qS', db['output_A'])])

*Define settings:*

In [7]:
s = GmsPy.GmsSettings(**{'name': name, 'db': db, 'groups': {g.name: g for g in (endo,exo)}})
s.Compile.run()

*Write blocks from trees:*

In [8]:
blocks = ''
for tree in A_tree.trees.values():
    blocks += getattr(_gamYProd,tree.f)(tree.name,tree.name)

*Add settings:*

In [9]:
s['g_endo'] = [endo.name]
s['g_exo'] = [exo.name]
s['blocks'] = [f"B_{tree.name}" for tree in A_tree.trees.values()]

*Write arguments and write:*

In [10]:
%%capture
s['args'] = s.stdArgs(blocks=blocks)
s.write()

In [11]:
m = GmsPy.GmsModel(ws=d['work'],**{'cns': 'CONOPT4'}) # use CONOPT4 to solve CNS models.
m.addlocal(db.name, db_str)
m.run(run = '\n'.join(s['text'].values()))